In [ ]:
!npm install -g localtunnel --no-fund
!npm audit fix --force
!pip install flask-cors requests

import zipfile
import os
from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import json
import threading
import shutil
import re
import time
import requests

app = Flask(__name__)
CORS(app)  # Permitir solicitudes CORS
UPLOAD_FOLDER = '/content/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
ZIP_FOLDER = '/content/zipfiles'  # Carpeta donde se almacenarán los archivos ZIP
os.makedirs(ZIP_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/', methods=['GET', 'OPTIONS'])
def home():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    return "Server is running", 200

@app.route('/upload', methods=['POST', 'OPTIONS'])
def upload_file():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    if 'file' not in request.files or 'project_name' not in request.form or 'ud_name' not in request.form:
        return 'No file part, project name, or unit name', 400
    file = request.files['file']
    project_name = request.form['project_name']
    ud_name = request.form['ud_name']
    if file.filename == '':
        return 'No selected file', 400
    if file:
        project_folder = os.path.join(app.config['UPLOAD_FOLDER'], project_name)
        ud_folder = os.path.join(project_folder, ud_name)
        os.makedirs(ud_folder, exist_ok=True)
        file_path = os.path.join(ud_folder, file.filename)
        file.save(file_path)
        process_uploaded_file(file_path, ud_folder)  # Llamar a la función para procesar el archivo
        return 'File uploaded and processed successfully', 200

@app.route('/rename_project', methods=['POST', 'OPTIONS'])
def rename_project():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    data = request.json
    old_name = data.get('old_name')
    new_name = data.get('new_name')
    if not old_name or not new_name:
        return 'Missing old or new project name', 400
    old_path = os.path.join(app.config['UPLOAD_FOLDER'], old_name)
    new_path = os.path.join(app.config['UPLOAD_FOLDER'], new_name)
    if os.path.exists(old_path):
        shutil.move(old_path, new_path)
        return 'Project renamed successfully', 200
    else:
        return 'Old project name not found', 404

@app.route('/rename_ud', methods=['POST', 'OPTIONS'])
def rename_ud():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    data = request.json
    project_name = data.get('project_name')
    old_name = data.get('old_name')
    new_name = data.get('new_name')
    if not project_name or not old_name or not new_name:
        return 'Missing project name, old name or new name', 400
    project_path = os.path.join(app.config['UPLOAD_FOLDER'], project_name)
    old_path = os.path.join(project_path, old_name)
    new_path = os.path.join(project_path, new_name)
    if os.path.exists(old_path):
        shutil.move(old_path, new_path)
        return 'Unit renamed successfully', 200
    else:
        return 'Old unit name not found', 404

@app.route('/delete_ud', methods=['POST', 'OPTIONS'])
def delete_ud():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    data = request.json
    project_name = data.get('project_name')
    ud_name = data.get('ud_name')
    if not project_name or not ud_name:
        return 'Missing project name or unit name', 400
    project_path = os.path.join(app.config['UPLOAD_FOLDER'], project_name)
    ud_path = os.path.join(project_path, ud_name)
    if os.path.exists(ud_path):
        shutil.rmtree(ud_path)
        return 'Unit deleted successfully', 200
    else:
        return 'Unit name not found', 404

@app.route('/delete_project', methods=['POST', 'OPTIONS'])
def delete_project():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    data = request.get_json()
    project_name = data['project_name']
    project_folder = os.path.join(app.config['UPLOAD_FOLDER'], project_name)
    if os.path.exists(project_folder):
        import shutil
        shutil.rmtree(project_folder)
        return 'Project deleted successfully', 200
    else:
        return 'Project not found', 404

@app.route('/check_auth', methods=['GET', 'OPTIONS'])
def check_auth():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    return jsonify({"auth_status": "success"}), 200

def build_cors_preflight_response():
    response = jsonify({"message": "CORS preflight"})
    response.headers.add("Access-Control-Allow-Origin", "*")
    response.headers.add("Access-Control-Allow-Headers", "*")
    response.headers.add("Access-Control-Allow-Methods", "GET, POST, OPTIONS, PUT, DELETE")
    return response

def process_uploaded_file(zip_path, extract_dir):
    os.makedirs(extract_dir, exist_ok=True)

    # Extraer archivo zip
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    # Listar archivos extraídos
    extracted_files = os.listdir(extract_dir)
    print(extracted_files)

@app.route('/uploads/<path:path>')
def serve_file(path):
    return send_from_directory(UPLOAD_FOLDER, path)

@app.route('/modify', methods=['POST', 'OPTIONS'])
def modify_index():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    project_name = request.json.get('project_name')
    ud_name = request.json.get('ud_name')
    new_value = request.json.get('ntxCafCompressed')

    if not project_name or not ud_name:
        return jsonify({"status": "project_name or ud_name is missing"}), 400

    index_path = os.path.join(app.config['UPLOAD_FOLDER'], project_name, ud_name, 'index_prof.html')

    # Añadir más información de depuración
    print(f"Project Name: {project_name}")
    print(f"Unit Name: {ud_name}")
    print(f"Index Path: {index_path}")

    if os.path.exists(index_path):
        with open(index_path, 'r') as index_file:
            content = index_file.read()

        # Usar una expresión regular para reemplazar la variable ntxCafCompressed
        updated_content = re.sub(r'var ntxCafCompressed = ".*";', f'var ntxCafCompressed = "{new_value}";', content)

        with open(index_path, 'w') as index_file:
            index_file.write(updated_content)

        return jsonify({"status": "success"})
    else:
        return jsonify({"status": "index file not found"}), 404

@app.route('/view_index', methods=['GET', 'OPTIONS'])
def view_index():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    project_name = request.args.get('project_name')
    ud_name = request.args.get('ud_name')

    if not project_name or not ud_name:
        return 'Missing project_name or ud_name', 400

    index_path = os.path.join(app.config['UPLOAD_FOLDER'], project_name, ud_name, 'index_prof.html')

    if os.path.exists(index_path):
        with open(index_path, 'r') as index_file:
            content = index_file.read()
        return content, 200
    else:
        return 'index_prof.html not found', 404

@app.route('/list_projects', methods=['GET', 'OPTIONS'])
def list_projects():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()

    projects = []
    if os.path.exists(UPLOAD_FOLDER):
        projects = os.listdir(UPLOAD_FOLDER)

    return jsonify({"projects": projects}), 200

@app.route('/get_ntxCafCompressed', methods=['GET', 'OPTIONS'])
def get_ntxCafCompressed():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()

    project_name = request.args.get('project_name')
    ud_name = request.args.get('ud_name')

    if not project_name or not ud_name:
        return jsonify({"error": "Missing project_name or ud_name"}), 400

    index_path = os.path.join(app.config['UPLOAD_FOLDER'], project_name, ud_name, 'index_prof.html')

    if not os.path.exists(index_path):
        return jsonify({"error": "index_prof.html not found"}), 404

    try:
        with open(index_path, 'r', encoding='utf-8') as index_file:
            content = index_file.read()

            # Usar una expresión regular para extraer el valor de ntxCafCompressed
            match = re.search(r'var ntxCafCompressed = "(.*?)";', content)
            if match:
                ntxCafCompressed_value = match.group(1)
                return jsonify({"ntxCafCompressed": ntxCafCompressed_value}), 200
            else:
                return jsonify({"error": "ntxCafCompressed variable not found"}), 404
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/list_ud_names', methods=['GET', 'OPTIONS'])
def list_ud_names():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()

    project_name = request.args.get('project_name')
    if not project_name:
        return jsonify({"error": "Missing project_name"}), 400

    project_path = os.path.join(app.config['UPLOAD_FOLDER'], project_name)
    if not os.path.exists(project_path):
        return jsonify({"error": "Project not found"}), 404

    # Listar las carpetas dentro del proyecto (que representan las UDs)
    ud_names = [d for d in os.listdir(project_path) if os.path.isdir(os.path.join(project_path, d))]
    return jsonify({"ud_names": ud_names}), 200

@app.route('/create_projects_zip', methods=['POST'])
def create_projects_zip():
    zip_filename = os.path.join(ZIP_FOLDER, 'projects.zip')

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zf:
        base_dir = UPLOAD_FOLDER
        for foldername in os.listdir(base_dir):
            folder_path = os.path.join(base_dir, foldername)
            if os.path.isdir(folder_path):
                for dirname, subdirs, files in os.walk(folder_path):
                    for filename in files:
                        file_path = os.path.join(dirname, filename)
                        zf.write(file_path, os.path.relpath(file_path, base_dir))

    # Responder a la aplicación que el ZIP ha sido creado
    return jsonify({"status": "success", "message": "Archivo ZIP creado exitosamente.", "zip_path": zip_filename})

@app.route('/download_projects_zip', methods=['GET'])
def download_projects_zip():
    zip_filename = os.path.join(ZIP_FOLDER, 'projects.zip')

    if os.path.exists(zip_filename):
        return send_file(zip_filename, mimetype='application/zip', as_attachment=True, download_name='projects.zip')
    else:
        return jsonify({"status": "error", "message": "El archivo ZIP no existe."}), 404


@app.route('/upload_zip', methods=['POST'])
def upload_zip():
    if 'file' not in request.files:
        return 'No file part', 400

    file = request.files['file']
    if file.filename == '':
        return 'No selected file', 400

    if file and file.filename.endswith('.zip'):
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)

        # Descomprimir el archivo ZIP en el servidor
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(app.config['UPLOAD_FOLDER'])

        # Eliminar el archivo ZIP una vez descomprimido
        os.remove(file_path)

        return 'File uploaded and extracted successfully', 200

    return 'Invalid file type', 400

@app.route('/create_ud_zip', methods=['POST'])
def create_ud_zip():
    data = request.json
    project_name = data.get('project_name')
    ud_name = data.get('ud_name')

    zip_filename = os.path.join(ZIP_FOLDER, f'{ud_name}.zip')

    ud_dir = os.path.join(UPLOAD_FOLDER, project_name, ud_name)
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zf:
        for dirname, subdirs, files in os.walk(ud_dir):
            for filename in files:
                file_path = os.path.join(dirname, filename)

                # Adjust the relative path to include the UD name as a folder inside the zip
                arcname = os.path.join(ud_name, os.path.relpath(file_path, ud_dir))

                zf.write(file_path, arcname)

    return jsonify({"status": "success", "message": "Archivo ZIP creado exitosamente.", "zip_path": zip_filename})

@app.route('/download_ud_zip', methods=['GET'])
def download_ud_zip():
    ud_name = request.args.get('ud_name')
    zip_filename = os.path.join(ZIP_FOLDER, f'{ud_name}.zip')

    if os.path.exists(zip_filename):
        return send_file(zip_filename, mimetype='application/zip', as_attachment=True, download_name=f'{ud_name}.zip')
    else:
        return jsonify({"status": "error", "message": "El archivo ZIP no existe."}), 404

@app.route('/upload_ud_zip', methods=['POST', 'OPTIONS'])
def upload_ud_zip():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()

    if 'file' not in request.files or 'project_name' not in request.form:
        return 'No file part or project name', 400

    file = request.files['file']
    project_name = request.form['project_name']

    if file.filename == '':
        return 'No selected file', 400

    if file and file.filename.endswith('.zip'):
        project_folder = os.path.join(app.config['UPLOAD_FOLDER'], project_name)
        os.makedirs(project_folder, exist_ok=True)

        # Guardar el archivo ZIP en el directorio del proyecto
        file_path = os.path.join(project_folder, file.filename)
        file.save(file_path)

        # Descomprimir el archivo ZIP dentro del directorio del proyecto
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(project_folder)

        # Eliminar el archivo ZIP una vez descomprimido
        os.remove(file_path)

        return 'UD ZIP file uploaded and extracted successfully', 200

    return 'Invalid file type', 400

@app.route('/upload_image', methods=['POST', 'OPTIONS'])
def upload_image():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    if 'file' not in request.files or 'project_name' not in request.form or 'ud_name' not in request.form:
        return 'No file part, project name, or unit name', 400
    file = request.files['file']
    project_name = request.form['project_name']
    ud_name = request.form['ud_name']
    if file.filename == '':
        return 'No selected file', 400
    if file:
        project_folder = os.path.join(app.config['UPLOAD_FOLDER'], project_name)
        ud_folder = os.path.join(project_folder, ud_name, 'resources')  # Guardar en carpeta 'resources'
        os.makedirs(ud_folder, exist_ok=True)
        file_path = os.path.join(ud_folder, file.filename)
        file.save(file_path)
        return 'Image uploaded successfully', 200

@app.route('/list_images', methods=['GET', 'OPTIONS'])
def list_images():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()

    project_name = request.args.get('project_name')
    ud_name = request.args.get('ud_name')

    if not project_name or not ud_name:
        return jsonify({"error": "Missing project_name or unit name"}), 400

    resources_folder = os.path.join(app.config['UPLOAD_FOLDER'], project_name, ud_name, 'resources')
    if not os.path.exists(resources_folder):
        return jsonify({"images": []}), 200

    # Listar todos los archivos de imágenes en el directorio de recursos
    valid_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp'}
    images = [f for f in os.listdir(resources_folder) if os.path.isfile(os.path.join(resources_folder, f)) and os.path.splitext(f)[1].lower() in valid_extensions]
    image_urls = [f'/uploads/{project_name}/{ud_name}/resources/{image}' for image in images]

    return jsonify({"images": image_urls}), 200

@app.route('/delete_image', methods=['POST', 'OPTIONS'])
def delete_image():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()

    data = request.json
    project_name = data.get('project_name')
    ud_name = data.get('ud_name')
    file_name = data.get('file_name')

    if not project_name or not ud_name or not file_name:
        return jsonify({"error": "Missing project_name, ud_name, or file_name"}), 400

    file_path = os.path.join(app.config['UPLOAD_FOLDER'], project_name, ud_name, 'resources', file_name)

    if os.path.exists(file_path):
        os.remove(file_path)
        return jsonify({"status": "success", "message": "Image deleted successfully"}), 200
    else:
        return jsonify({"status": "error", "message": "Image not found"}), 404


def run_app():
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=run_app)
thread.start()

time.sleep(5)  # Espera a que localtunnel se inicie

# Obtener la contraseña del túnel usando requests
response = requests.get('https://loca.lt/mytunnelpassword')
tunnel_password = response.text.strip()
print(f"Tunnel password: {tunnel_password}")

!lt --port 5000



added 22 packages, and audited 23 packages in 5s

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
npm WARN using --force Recommended protections disabled.
npm ERR! code ENOLOCK
npm ERR! audit This command requires an existing lockfile.
npm ERR! audit Try creating one first with: npm i --package-lock-only
npm ERR! audit Original error: loadVirtual requires existing shrinkwrap file

npm ERR! A complete log of this run can be found in:
npm ERR!     /root/.npm/_logs/2024-08-21T10_38_26_714Z-debug-0.log
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Tunnel password: 34.48.212.140
your url is: https://orange-taxis-sin.loca.lt


INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:38:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:38:54] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:39:07] "GET /check_auth HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:39:08] "OPTIONS /list_projects HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:39:08] "GET /list_projects HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:40:22] "OPTIONS /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:40:45] "POST /upload HTTP/1.1" 200 -


['components.json', 'ud_files.zip', 'index_prof.html', 'resources', 'trackingConfig.js', 'project.json', 'cmi5.json', '573_215_0_532']


...

Project Name: Nuevo Proyecto 1
Unit Name: Unidad Didáctica 1
Index Path: /content/uploads/Nuevo Proyecto 1/Unidad Didáctica 1/index_prof.html


INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:50:19] "OPTIONS /view_index?project_name=Nuevo%20Proyecto%201&ud_name=Unidad%20Didáctica%201 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:50:19] "GET /view_index?project_name=Nuevo%20Proyecto%201&ud_name=Unidad%20Didáctica%201 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:50:20] "GET /uploads/Nuevo%20Proyecto%201/Unidad%20Didáctica%201/index_prof.html?no_cache=1724237419725 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:50:20] "GET /uploads/Nuevo%20Proyecto%201/Unidad%20Didáctica%201/trackingConfig.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:50:20] "GET /uploads/Nuevo%20Proyecto%201/Unidad%20Didáctica%201/573_215_0_532/loader.js?random=0.341097069207015 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:50:22] "GET /uploads/Nuevo%20Proyecto%201/Unidad%20Didáctica%201/573_215_0_532/mathlive/mathlive.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:50:22] "GET /

Project Name: Nuevo Proyecto 1
Unit Name: Unidad Didáctica 1
Index Path: /content/uploads/Nuevo Proyecto 1/Unidad Didáctica 1/index_prof.html


INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:51:01] "GET /view_index?project_name=Nuevo%20Proyecto%201&ud_name=Unidad%20Didáctica%201 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:51:01] "GET /uploads/Nuevo%20Proyecto%201/Unidad%20Didáctica%201/index_prof.html?no_cache=1724237461202 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:51:01] "GET /uploads/Nuevo%20Proyecto%201/Unidad%20Didáctica%201/trackingConfig.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:51:01] "GET /uploads/Nuevo%20Proyecto%201/Unidad%20Didáctica%201/573_215_0_532/loader.js?random=0.5342051498492377 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:51:02] "GET /uploads/Nuevo%20Proyecto%201/Unidad%20Didáctica%201/573_215_0_532/mathlive/mathlive.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:51:02] "GET /uploads/Nuevo%20Proyecto%201/Unidad%20Didáctica%201/573_215_0_532/ckeditor/ckeditor.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 10:51:03

^C
